In [33]:
# Loading packages
import numpy as np
import sdeint
import deepdish.io as dd
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt
from scipy import signal
import matplotlib.gridspec as gridspec
%matplotlib inline
import time
from IPython.display import set_matplotlib_formats
from ipywidgets import interact, interactive,FloatSlider,IntSlider,ToggleButton
from ipywidgets import Button, HBox, VBox, Text,Select,Layout,Dropdown,interactive_output
set_matplotlib_formats('pdf', 'png')
plt.rcParams['savefig.dpi'] = 75

plt.rcParams['figure.autolayout'] = False
plt.rcParams['figure.figsize'] = 10, 6
plt.rcParams['axes.labelsize'] = 18
plt.rcParams['axes.titlesize'] = 20
plt.rcParams['font.size'] = 16
plt.rcParams['lines.linewidth'] = 2.0
plt.rcParams['lines.markersize'] = 8
plt.rcParams['legend.fontsize'] = 14

plt.rcParams['text.usetex'] = True
plt.rcParams['font.family'] = "sans"
plt.rcParams['font.serif'] = "cm"
#plt.rcParams['text.latex.preamble'] = "\usepackage{subdepth}, \usepackage{type1cm}"

In [26]:
# Setting up the model
from sympy import symbols,solve,Eq,simplify
from BenincaModel import BenincaModel,Es_normal
#Es_normal['rhs']='RM_forced'
Es_normal["verbose"]=False
m = BenincaModel(Es=Es_normal,Ps='auto/Beninca_set1.hdf5',Vs=None)
m.print_equations()
p=m.p.copy()
P_syms = m.Ps_symbols
V_syms = m.var_symbols
B0_0 = (P_syms['cBR']-P_syms['mB'])/P_syms['cBR']
print(p)

dB0dt =  -A*B0*cAB - B0*M*cM - B0*mB + BA*mA*(alpha*(Tmax - Tmean)*cos(0.01721420632104*t - 0.550854602273279) + 1.0) + cBR*(B0 + BA)*(-A - B0 - M + 1.0)
dBAdt =  A*B0*cAB - BA*M*cM - BA*mA*(alpha*(Tmax - Tmean)*cos(0.01721420632104*t - 0.550854602273279) + 1.0) - BA*mB
dAdt  =  A*B0*cAB - A*M*cM + A*cAR*(-A - B0 - M + 1.0) - A*mA*(alpha*(Tmax - Tmean)*cos(0.01721420632104*t - 0.550854602273279) + 1.0)
dMdt  =  M*cM*(A + B0) - M*mM*(alpha*(Tmax - Tmean)*cos(0.01721420632104*t - 0.550854602273279) + 1.0)
{'Tmax': 17.1, 'Tmean': 17.1, 'alpha': 1.0, 'cAB': 0.049, 'cAR': 0.021, 'cBR': 0.018, 'cM': 0.078, 'mA': 0.013, 'mB': 0.003, 'mM': 0.017, 'muAB': 0.036, 'muAR': 0.008, 'muB': 0.015, 'muM': 0.061}


In [29]:
jac_alpha0=m.sym_localJac.subs(P_syms['alpha'],0)

jac_alpha0_steady_state = jac_alpha0.subs([(V_syms['BA'],0),(V_syms['A'],0),(V_syms['M'],0),(V_syms['B0'],B0_0)])
print(simplify(jac_alpha0_steady_state))
eigs_dict = simplify(jac_alpha0_steady_state).eigenvals()
eigs = []
for key in eigs_dict:
    print(m.sub_parms(key))
    eigs.append(key)
print(eigs)

Matrix([[-cBR + mB, 1.0*mA + mB, -cAB + cAB*mB/cBR - cBR + mB, -cBR - cM + mB + cM*mB/cBR], [0, -1.0*mA - mB, cAB*(cBR - mB)/cBR, 0], [0, 0, (cAB*(cBR - mB) + cAR*mB - 1.0*cBR*mA)/cBR, 0], [0, 0, 0, cM - 1.0*mM - cM*mB/cBR]])
-0.0150000000000000
-0.0160000000000000
0.0313333333333333
0.0480000000000000
[-cBR + mB, -mA - mB, (cAB*cBR - cAB*mB + cAR*mB - cBR*mA)/cBR, (cBR*cM - cBR*mM - cM*mB)/cBR]


In [42]:
cAB = FloatSlider(min=0., max=2.0*m.p['cAB'], step=0.01, value=m.p['cAB'])
cAR = FloatSlider(min=0., max=2.0*m.p['cAR'], step=0.01, value=m.p['cAR'])
cM  = FloatSlider(min=0., max=2.0*m.p['cM'], step=0.01, value=m.p['cM'])
mM  = FloatSlider(min=0., max=2.0*m.p['mM'], step=0.01, value=m.p['mM'])
ui = HBox([cAB, cAR, cM, mM])
print(m.p['cAB'],m.p['cAR'],m.p['cM'],m.p['mM'])
def f(cAB, cAR, cM,mM):
    for eig in eigs:
        eigs_new=eig.subs([(P_syms['cBR'],m.p['cBR']),(P_syms['mB'],m.p['mB']),(P_syms['mA'],m.p['mA'])])
        print(eigs_new.subs([(P_syms['cAB'],cAB),(P_syms['cAR'],cAR),(P_syms['cM'],cM),(P_syms['mM'],mM)]))

out = interactive_output(f, {'cAB': cAB, 'cAR': cAR, 'cM': cM,'mM':mM})

display(ui, out)

0.049 0.021 0.078 0.017


Output()